## Visualizing results -  Original Geoparser

We have run the [Edinburgh geoparser](https://programminghistorian.org/en/lessons/geoparsing-text-with-edinburgh#patch-fix) to extract and resolvoe locations contained within the **First Edition, Volume 1, A-B** (year 1771) of the *Encyclopaedia Britannica*
                             
The query for running that we have used is the following: 

`spark-submit --py-files defoe.zip defoe/run_query.py nls_subsample.txt nls defoe.nls.queries.geoparser_pages -r geopaser_original_EB_144133901 -n 16`

The nls_subsample.txt has the path of this particular book that we are exploring:

`xxx/nls-data-encyclopaediaBritannica/144133901`

The results file ( [geopaser_original_EB_144133901](http://localhost:8888/edit/results_NLS/geopaser_original_EB_144133901)) is stored in this repository under the *results_NLS* subdirectory. 


### Loading the necessary libraries

In [8]:
import yaml
from IPython.core.display import display, HTML
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np

### Functions

In [9]:
def read_query_results(filename):
    with open('./results_NLS/'+filename, 'r') as f:
        query_results = yaml.load(f)
    return query_results


In [12]:
def geresolution_dataframe(result):
    dfs=[]
    data=[]
    c_locs=[]
    for i in result.keys():
        t_ind = 0
        e_ind = 0
        for k in result[i]:
            locs= k["georesolution_page"]
            page = k["text_unit id"]
       
            if locs != {}:
                data=[]
                l_ind = 0
                for i in locs:
                    if type(locs[i]) == type([]) :
                        c_locs=locs[i].copy()
                        c_locs.append(i.split("-")[0])
                        c_locs.append(page)
                        data.append(c_locs)
                        l_ind = l_ind + 1   
                e_ind = t_ind + l_ind 
                if data:
                    df_page = pd.DataFrame(data, columns = ['Latitude', 'Longitude', 'Place', 'Page'], 
                                      index=list(range(t_ind, e_ind)))
                    dfs.append(df_page)
                    t_ind=e_ind
    df_total = pd.concat(dfs)
    return df_total


This query does the following tasks:

- Ingests all the pages from the directory "144133901",  which corresponds to the book "Encyclopaedia Britannica; or, A dictionary of arts and sciences, compiled upon a new plan … - First edition, 1771, Volume 1, A-B - EB.1"
- Cleans the text applied two fixes: Long-s and hyphen words
- Identifies *entities* using the original geotagging of the geoparser. 
- From the previous entities, just selects the one about *location* and creates an xml (in memory) per page with these "location" entities. 
- Applies the georesolve to each xml and gets lat and long. **Important: Everything is in memory, we do not create XML files in those steps**
- Group the results by Book's title, and also gets some informative metadata

As a result we get a file per gazetter|book with an entry per page with the following information:

    * archive_filename: Path to the gazetteer 
    * clean_text: Page's clean text after applying 2 fixes: long-S and hyphenate words
    * display_ER: Display of a page’s entities found by the geoparser (in HTML format)
    * edition: Edition of the gazetteer
    * georesolution_page: Page's geolocations after applying the georesolver
    * model: defoe model – could be fmp|nls|papers|alto. In this case is “nls”
    * text_unit: page (for other defoe models could be “article”)
    * num_text_unit: number of tex units. In this case, number of pages of this particular gazetter (e.g. 832)
    * page_filename: Page's filename (page's relative path)
    * text_unit id: The number of this page (e.g. Page 1)
    * lang_model : The language model applied (original_geoparser)
    * type_distribution: type of document (newspaper|book). In this case is "book". 
    * year: Publication year 


Example:
  - archive_filename: /home/tdm/datasets/encyclopaedia-britannica-sample/144133901
  - clean_text: "AGRICULTURE. 47 PART II. Of the various Operations upon the Soil, in\
    \ order to prepare it for the Recsp-' tion and Nourijhment of Plants. S e c t.\
  - edition: First edition, 1771, Volume 1, A-B
  - georesolution_page:
     - Scotland-rb2:
       - '57.68633318560074'
       - '-4.96890721218449'
  - lang_model: geoparser_original
  - model: nls
  - num_text_unit: 832
  - page_filename: alto/188083661.34.xml
  - place: Edinburgh
  - text_unit: page
  - text_unit id: Page73
  - type_distribution: book



In [13]:
results=read_query_results('geopaser_original_EB_144133901')

In [14]:
df_total= geresolution_dataframe(results)

In [15]:
df_total

,Latitude,Longitude,Place,Page
0,57.68633318560074,-4.96890721218449,Scotland,Page73
1,51.66553761368909,0.6494482339095444,Maries,Page74
2,52.66970530850324,-1.117070990383866,England,Page74
3,53.69180622064813,-2.423529257645026,Scotland,Page74
4,51.66553761368909,0.6494482339095444,Maries,Page74
5,53.69180622064813,-2.423529257645026,Scotland,Page74
6,42.7764787,-71.1522811,Scotland,Page75
7,42.4514848,-71.0864414,England,Page75
8,57.68633318560074,-4.96890721218449,Scotland,Page76
9,67.3741216808404,12.1815410351753,Rost,Page77


In [16]:
df_total[["Place"]].count()

Place    5045
dtype: int64

In [17]:
df_total.sum()

Latitude     57.6863331856007451.6655376136890952.669705308...
Longitude    -4.968907212184490.6494482339095444-1.11707099...
Place        ScotlandMariesEnglandScotlandMariesScotlandSco...
Page         Page73Page74Page74Page74Page74Page74Page75Page...
dtype: object

In [18]:
df_total.groupby("Place").count()

,Latitude,Longitude,Page
Place,,,
ABACA,1,1,1
AGROPOLI,1,1,1
AGUALVA,1,1,1
AGUTI,1,1,1
AHAUSEN,1,1,1
AHUAS,1,1,1
AHUN,1,1,1
AHUS,1,1,1
AIGUILLON,1,1,1


In [19]:
df_total.groupby("Page").count()

,Latitude,Longitude,Place
Page,,,
Page100,9,9,9
Page101,37,37,37
Page102,25,25,25
Page103,34,34,34
Page104,24,24,24
Page105,34,34,34
Page106,42,42,42
Page109,22,22,22
Page110,23,23,23


In [23]:
df_total.groupby(["Latitude", "Longitude"]).count()

,,Place,Page
Latitude,Longitude,,
,,271,271
-0.1366818175840478,100.6383162769587,1,1
-0.1544928803232608,-78.44005994797827,1,1
-0.2333333,-78.33333330000001,1,1
-1.0753,121.7811,1,1
-1.15,17.3333333,1,1
-1.380613091643525,-48.41852484800316,1,1
-1.445725396559766,5.629326002193295,1,1
-10.28333,14.95,1,1
